In [1]:
import os
import random
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import re
import scipy.io
import pandas as pd

In [2]:
# TensorFlow settings and training parameters
AUTOTUNE = tf.data.experimental.AUTOTUNE
EPOCHS = 500
BATCH_SIZE = 16
PATIENCE = 10
LEARNING_RATE = 1e-3
IMAGE_SIZE = 299
INITIAL_EPOCH = 0

In [3]:
def load_image(is_labelled: bool, is_training=True):
  def _get_image(path: str) -> tf.Tensor:
    image = tf.image.decode_jpeg(tf.io.read_file(path), channels=3)
    image = tf.cast(image, dtype=tf.int32)
    image = tf.image.resize_with_pad(image, IMAGE_SIZE, IMAGE_SIZE)
    if is_training:
        image = tf.image.random_flip_left_right(image)
        image = tf.image.random_brightness(image, 0.1)
        image = tf.image.random_contrast(image, 0.1, 0.2)
        image = tf.image.random_saturation(image, 0.9, 1.1)
        image = tf.image.random_hue(image, 0.1)
    return tf.keras.applications.inception_resnet_v2.preprocess_input(image)

  def _get_image_label(img: tf.Tensor, label: int) -> tuple:
    return _get_image(img), label

  return _get_image_label if is_labelled else _get_image


In [4]:
def prepare_dataset(dataset, is_training=True, is_labeled=True):
  image_read_fn = load_image(is_labeled, is_training)
  dataset = dataset.map(image_read_fn, num_parallel_calls=AUTOTUNE)
  return dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)

In [5]:
def create_model() -> tf.keras.Model:
  feature_model = tf.keras.applications.InceptionResNetV2(
  include_top=False, pooling='avg')
  feature_model.trainable = False

  model = tf.keras.Sequential([
      tf.keras.Input((IMAGE_SIZE, IMAGE_SIZE, 3)),
      feature_model,
      tf.keras.layers.Dense(512, activation='selu'),
      tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
                loss=tf.keras.losses.MeanSquaredError(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  return model

In [6]:
def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [7]:
def reconstruct_path(image_id: int) -> str:
    image_id = str(image_id).rjust(6, '0')
    return f'drive/MyDrive/TIES4911/mini-project/frames/seq_{image_id}.jpg'

In [8]:
set_seed()

In [9]:
META_FILE = 'drive/MyDrive/TIES4911/mini-project/labels.csv'

In [10]:
data = pd.read_csv(META_FILE)
data['path'] = data['id'].apply(reconstruct_path)
data.head()

,id,count,path
0,1,35,drive/MyDrive/TIES4911/mini-project/frames/seq...
1,2,41,drive/MyDrive/TIES4911/mini-project/frames/seq...
2,3,41,drive/MyDrive/TIES4911/mini-project/frames/seq...
3,4,44,drive/MyDrive/TIES4911/mini-project/frames/seq...
4,5,41,drive/MyDrive/TIES4911/mini-project/frames/seq...


In [11]:
mat = scipy.io.loadmat('drive/MyDrive/TIES4911/mini-project/mall_gt.mat')
count = [[element for element in upperElement] for upperElement in mat['count']]
count = list(map(lambda x: x[0], count))
count

[29,
 30,
 35,
 31,
 26,
 26,
 28,
 27,
 29,
 31,
 37,
 37,
 41,
 41,
 41,
 40,
 42,
 44,
 36,
 37,
 38,
 37,
 40,
 41,
 37,
 41,
 41,
 41,
 36,
 33,
 35,
 36,
 37,
 36,
 35,
 36,
 33,
 31,
 30,
 29,
 27,
 24,
 25,
 21,
 22,
 19,
 17,
 17,
 18,
 17,
 17,
 16,
 16,
 17,
 16,
 19,
 22,
 22,
 20,
 21,
 23,
 18,
 23,
 23,
 22,
 24,
 24,
 25,
 20,
 23,
 26,
 26,
 25,
 23,
 22,
 20,
 16,
 18,
 17,
 17,
 16,
 15,
 15,
 17,
 16,
 16,
 16,
 14,
 15,
 13,
 18,
 18,
 16,
 18,
 20,
 19,
 24,
 25,
 27,
 31,
 31,
 30,
 31,
 33,
 33,
 33,
 31,
 25,
 23,
 20,
 23,
 23,
 22,
 22,
 24,
 24,
 26,
 27,
 29,
 25,
 24,
 26,
 25,
 24,
 26,
 31,
 32,
 28,
 29,
 29,
 30,
 29,
 27,
 29,
 28,
 25,
 27,
 26,
 27,
 25,
 22,
 23,
 23,
 26,
 23,
 25,
 22,
 19,
 22,
 24,
 23,
 21,
 20,
 20,
 20,
 23,
 24,
 24,
 23,
 20,
 20,
 23,
 26,
 26,
 27,
 26,
 23,
 23,
 22,
 21,
 19,
 19,
 18,
 18,
 16,
 15,
 17,
 17,
 18,
 19,
 20,
 17,
 19,
 23,
 23,
 25,
 25,
 29,
 29,
 30,
 29,
 23,
 22,
 21,
 21,
 21,
 22,
 24,
 25,
 27,


In [12]:
# Create train and validation data sets.
data_train = data.head(1700)
data_valid = data.tail(300)

ds_train = tf.data.Dataset.from_tensor_slices((data_train['path'], count[:1700]))
ds_valid = tf.data.Dataset.from_tensor_slices((data_valid['path'], count[1700:]))

ds_train = prepare_dataset(ds_train)
ds_valid = prepare_dataset(ds_valid, is_training=False)

In [13]:
checkpoint_path = "drive/MyDrive/TIES4911/mini-project/training-2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

In [14]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'drive/MyDrive/TIES4911/mini-project/training-2/cp-0022.ckpt'

In [15]:
model = create_model()

if latest:
  model.load_weights(latest)
  INITIAL_EPOCH = int(re.findall(r".cp-(\d{4})\.ckpt", latest)[0])
else:
  INITIAL_EPOCH = 0

model.save_weights(checkpoint_path.format(epoch=0))

219055592/219055592 [==============================] - 2s 0us/step


In [17]:
INITIAL_EPOCH

22

In [ ]:
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*BATCH_SIZE)

early_stop = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', patience=PATIENCE,
    restore_best_weights=True)

lr_reduction = tf.keras.callbacks.ReduceLROnPlateau(
    monitor='val_loss', patience=1, cooldown=1, verbose=1,
    factor=0.75, min_lr=1e-8)

history = model.fit(ds_train, validation_data=ds_valid, batch_size=BATCH_SIZE,
                    initial_epoch=INITIAL_EPOCH, epochs=EPOCHS, callbacks=[cp_callback, early_stop, lr_reduction],
                    use_multiprocessing=True, workers=os.cpu_count())

Epoch 23/500
 79/107 [=====================>........] - ETA: 4:34 - loss: 23.4530 - mean_absolute_error: 3.9103
Epoch 23: saving model to drive/MyDrive/TIES4911/mini-project/training-2/cp-0023.ckpt
107/107 [==============================] - 1281s 12s/step - loss: 22.5944 - mean_absolute_error: 3.8327 - val_loss: 14.9884 - val_mean_absolute_error: 3.0497 - lr: 1.0011e-04
Epoch 24/500
 52/107 [=============>................] - ETA: 9:12 - loss: 23.0137 - mean_absolute_error: 3.8562
Epoch 24: saving model to drive/MyDrive/TIES4911/mini-project/training-2/cp-0024.ckpt
107/107 [==============================] - 1249s 12s/step - loss: 21.2077 - mean_absolute_error: 3.7008 - val_loss: 14.9132 - val_mean_absolute_error: 3.0434 - lr: 1.0011e-04
Epoch 25/500
 25/107 [======>.......................] - ETA: 12:46 - loss: 24.3709 - mean_absolute_error: 4.0011
Epoch 25: saving model to drive/MyDrive/TIES4911/mini-project/training-2/cp-0025.ckpt
105/107 [============================>.] - ETA: 18s - l

In [ ]:
mse, mae = model.evaluate(ds_valid)
print(f'Validation MSE = {mse}\n'
      f'Validation MAE = {mae}')

In [ ]:
model.save('drive/MyDrive/TIES4911/mini-project/model.h5')